In [1]:
!pip install python-geohash
!pip install pygeohash
!pip install geopy
!pip install seaborn
!pip install statsmodels
!pip install esda
!pip install libpysal
!pip install linearmodels

  Using cached python_geohash-0.8.5-cp39-cp39-linux_x86_64.whl
  Using cached pygeohash-3.2.0-cp39-cp39-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl (43 kB)
  Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
  Using cached geographiclib-2.1-py3-none-any.whl (40 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached statsmodels-0.14.5-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (10.7 MB)
  Using cached patsy-1.0.2-py2.py3-none-any.whl (233 kB)
  Using cached esda-2.5.1-py3-none-any.whl (132 kB)
  Using cached libpysal-4.8.1-py3-none-any.whl (2.8 MB)
  Using cached linearmodels-6.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 26.8 MB/s eta 0:00:00:00:01
  Using cached setuptools_scm-8.3.1-py3-none-any.whl (43 kB)
  Using cached pyhdfe-0.2.0-py3-none-any.whl (19 kB)
  Using cached formulaic-1.2.1-py3-none-any.whl (117 kB)
  Using cached

In [2]:
import warnings
from collections import Counter

warnings.filterwarnings("ignore")

In [3]:
from read_bothdecision import *
from compare_geohash import *
from more_grained_analysis import *

# bbox=(-82.0, 26.48, -81.92, 26.52)
bbox=(-82.04150698533131, 26.490448860026532, -81.87502336164845, 26.604200914701607)
# bbox=(-81.98048446650292,26.487614477169117,-81.93348521668332,26.52132073874843)
min_lon, min_lat, max_lon, max_lat = bbox
decision_df, households_df = load_decision_data(bbox=bbox)
network_dict = load_network_files('results',bbox=bbox)
result = compare_geohash_overlap(decision_df, households_df, network_dict)
network_dict, mapping=fast_match_network_to_households(households_df, network_dict)
result = compare_geohash_overlap(decision_df, households_df, network_dict)
damage_df=pd.read_csv('hurricane_ian_damage_filtered2.csv')
damage_df = damage_df[ (damage_df['Longitude'] >= min_lon) & (damage_df['Longitude'] <= max_lon) 
                        & (damage_df['Latitude'] >= min_lat) & (damage_df['Latitude'] <= max_lat) ]
damage_df = compute_geohash_overlap(damage_df, households_df, precision=8)

[INFO] results/Group_social_network_2022-08-01.csv: kept 124875 edges, 7817 nodes in bbox
[INFO] results/Group_social_network_2022-09-01.csv: kept 76571 edges, 6557 nodes in bbox
[INFO] results/Group_social_network_2022-10-01.csv: kept 139558 edges, 10046 nodes in bbox
[INFO] results/Group_social_network_2022-11-01.csv: kept 129182 edges, 8756 nodes in bbox
[INFO] results/Group_social_network_2022-12-01.csv: kept 150352 edges, 9135 nodes in bbox
[INFO] results/Group_social_network_2023-01-01.csv: kept 2050 edges, 993 nodes in bbox
[INFO] results/Group_social_network_2023-02-01.csv: kept 1739 edges, 931 nodes in bbox
[INFO] results/Group_social_network_2023-03-01.csv: kept 1221 edges, 767 nodes in bbox
[INFO] results/Group_social_network_2023-04-01.csv: kept 1200 edges, 737 nodes in bbox
[INFO] results/Group_social_network_2023-05-01.csv: kept 1255 edges, 765 nodes in bbox
[INFO] results/Group_social_network_2023-06-01.csv: kept 1388 edges, 825 nodes in bbox
[INFO] results/Group_social_

In [4]:
household_value=pd.read_csv('decision_data/filtered_geohash.csv')

In [5]:
import pandas as pd

def build_analysis_table(damage_df, households_df, household_value, network_dict):
    """
    Build a clean analysis table for social contagion IV design.
    Adds filtering so that only edges where exactly one household is damaged are kept.

    Steps:
      1) Union all monthly social networks (undirected unique edges)
      2) Merge damage, building, and land info
      3) Filter edges: keep only pairs where exactly ONE side has DamageLevel_num > 0
      4) Compute peer averages based on filtered network
      5) Merge self and peer features
    """

    # ---------- 1. Union all monthly networks ----------
    all_edges = []
    for _, df in network_dict.items():
        e = df[['group_1', 'group_2']].rename(columns={'group_1':'node_a','group_2':'node_b'})
        all_edges.append(e)
    edges = pd.concat(all_edges, ignore_index=True)
    # make undirected, deduplicate
    edges = pd.concat([edges, edges.rename(columns={'node_a':'node_b','node_b':'node_a'})],
                      ignore_index=True).drop_duplicates()

    # ---------- 2. Build base household table ----------
    base = households_df[['geohash8','has_repair_decision','has_sales_decision']].copy()
    base['has_decision'] = (base['has_repair_decision'] | base['has_sales_decision']).astype(int)

    # attach damage info
    dmg = damage_df[['damage_geohash','DamageLevel']].rename(columns={'damage_geohash':'geohash8'})
    base = base.merge(dmg, on='geohash8', how='left')

    # numeric damage level
    damage_map = {'None':0, 'Affected':1, 'Minor':2, 'Major':3, 'Destroyed':4}
    base['DamageLevel_num'] = base['DamageLevel'].map(damage_map).fillna(0).astype(int)

    # attach building / land value info
    val = household_value.rename(columns={
        'geohash':'geohash8',
        'Building Value ($)':'BldgValue',
        'Land Value ($)':'LandValue'
    })[['geohash8','BldgValue','LandValue']]
    base = base.merge(val, on='geohash8', how='left')

    # compact self attributes
    self_df = base[['geohash8','has_decision','DamageLevel_num','BldgValue','LandValue']]

    # ---------- 3. Filter edges: keep only pairs where exactly one side damaged ----------
    # attach each node's damage level
    edges = edges.merge(
        self_df[['geohash8', 'DamageLevel_num']],
        left_on='node_a', right_on='geohash8', how='left'
    ).rename(columns={'DamageLevel_num': 'damage_a'}).drop(columns='geohash8')

    edges = edges.merge(
        self_df[['geohash8', 'DamageLevel_num']],
        left_on='node_b', right_on='geohash8', how='left'
    ).rename(columns={'DamageLevel_num': 'damage_b'}).drop(columns='geohash8')

    # keep only pairs where exactly one side is damaged
    edges = edges[
        ((edges['damage_a'] > 0) & (edges['damage_b'] == 0)) |
        ((edges['damage_a'] == 0) & (edges['damage_b'] > 0))
    ].drop(columns=['damage_a','damage_b'])

    # ---------- 4. Compute peer averages ----------
    nbr = edges.merge(self_df, left_on='node_b', right_on='geohash8', how='left')

    peer_avg = (
        nbr.groupby('node_a')
           .agg(peer_avg_decision=('has_decision','mean'),
                peer_avg_damage=('DamageLevel_num','mean'),
                peer_avg_BldgValue=('BldgValue','mean'),
                peer_avg_LandValue=('LandValue','mean'),
                peer_degree=('node_b','count'))
           .reset_index()
           .rename(columns={'node_a':'geohash8'})
    )

    # ---------- 5. Merge self and peer features ----------
    analysis_df = self_df.merge(peer_avg, on='geohash8', how='left')
    for c in ['peer_avg_decision','peer_avg_damage','peer_avg_BldgValue','peer_avg_LandValue','peer_degree']:
        analysis_df[c] = analysis_df[c].fillna(0)

    return analysis_df


In [6]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

def first_stage_regression(analysis_df):
    """
    First-stage OLS regression for IV design:
        peer_avg_decision = λ * peer_avg_damage
                            + ψ0 * (my own controls)
                            + γ1 * (peer characteristics)
                            + ε
    """

    # ---------- Step 1: prepare data ----------
    cols_needed = ['peer_avg_decision', 'peer_avg_damage',
                   'peer_avg_BldgValue', 'peer_avg_LandValue',
                   'DamageLevel_num', 'BldgValue']
    df = analysis_df[cols_needed].copy().dropna()

    # scale large values for stability
    df['peer_avg_BldgValue_scaled'] = df['peer_avg_BldgValue'] / 1e5
    df['peer_avg_LandValue_scaled'] = df['peer_avg_LandValue'] / 1e5
    df['BldgValue_scaled'] = df['BldgValue'] / 1e5

    # ---------- Step 2: fit OLS ----------
    y = df['peer_avg_decision']
    X = df[['peer_avg_damage',
            'peer_avg_BldgValue_scaled', 'peer_avg_LandValue_scaled',
            'DamageLevel_num', 'BldgValue_scaled']]
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()

    # ---------- Step 3: output ----------
    results_table = pd.DataFrame({
        'Variable': model.params.index,
        'Coef': model.params.values,
        'StdErr': model.bse.values,
        't': model.tvalues.values,
        'p_value': [f"{p:.8f}" for p in model.pvalues.values]
    })

    f_stat = model.f_test("peer_avg_damage = 0")

    print("\n================= First-Stage Regression Results (OLS) =================")
    print(results_table.to_string(index=False))
    print("=======================================================================")
    print(f"F-statistic for instrument (peer_avg_damage): {float(f_stat.fvalue):.3f}")
    print(f"p-value: {float(f_stat.pvalue):.8f}")
    print("=======================================================================")

    return model


In [7]:
import pandas as pd
from linearmodels.iv import IV2SLS

def second_stage_regression(analysis_df, first_stage_model):
    """
    Run the second-stage IV regression (2SLS) for social contagion analysis.
    Output format and naming consistent with the first-stage OLS table.
    """

    # --- Step 1: predicted peer decisions from first stage ---
    analysis_df = analysis_df.copy()
    analysis_df['peer_avg_decision_hat'] = first_stage_model.fittedvalues

    # --- Step 2: clean data ---
    cols = ['has_decision', 'peer_avg_decision', 'peer_avg_damage',
            'BldgValue', 'DamageLevel_num', 'peer_avg_BldgValue', 'peer_avg_LandValue']
    df = analysis_df[cols].dropna()

    # --- Step 3: run IV regression ---
    model = IV2SLS.from_formula(
        'has_decision ~ 1 + BldgValue + DamageLevel_num + peer_avg_BldgValue + peer_avg_LandValue '
        '[peer_avg_decision ~ peer_avg_damage]',
        data=df
    ).fit(cov_type='robust')

    # --- Step 4: construct clean table ---
    results_table = pd.DataFrame({
        'Variable': model.params.index,
        'Coef': model.params.values,
        'StdErr': model.std_errors.values,
        't': model.tstats.values,
        'p_value': [f"{p:.8f}" for p in model.pvalues.values]
    })

    # rename Intercept to const for consistency
    results_table['Variable'] = results_table['Variable'].replace({'Intercept': 'const'})

    print("\n================= Second-Stage Regression Results (IV-2SLS) =================")
    print(results_table.to_string(index=False))
    print("===========================================================================")
    print(f"F-statistic (overall): {model.f_statistic.stat:.3f}")
    print(f"p-value: {model.f_statistic.pval:.8f}")
    print("===========================================================================")

    return model


In [8]:
analysis_df=build_analysis_table(damage_df, households_df, household_value, network_dict)

In [32]:
sample_df = analysis_df[analysis_df['peer_avg_BldgValue'] != 0].sample(n=10, random_state=42)
sample_df

,geohash8,has_decision,DamageLevel_num,BldgValue,LandValue,peer_avg_decision,peer_avg_damage,peer_avg_BldgValue,peer_avg_LandValue,peer_degree
61622,dhtxpfq4,1,1,2.234030e+05,0.0,0.561471,0.000000,3.011720e+05,5.570802e+03,32023.0
6979,dhtwy6hf,0,0,1.779434e+06,243227.0,0.923077,1.974359,2.736871e+05,8.310624e+04,78.0
80673,dhtyb4e5,1,2,3.200750e+05,63793.0,0.534247,0.000000,2.897268e+05,1.087196e+05,292.0
21770,dhtwz8su,0,3,1.346910e+05,65408.0,0.983673,0.000000,1.921742e+05,1.064949e+05,735.0
13822,dhtwz1u7,1,2,2.024443e+05,0.0,0.000000,0.000000,4.062276e+06,8.831851e+06,12.0
109450,dhtz2bg5,0,0,1.711160e+05,46028.0,0.965753,1.924658,1.104067e+05,7.226515e+04,292.0
58525,dhtxp5hy,1,0,1.037190e+05,0.0,0.000000,2.000000,2.956620e+05,9.044000e+04,9.0
4519,dhtwxpgu,1,0,1.536415e+05,0.0,0.000000,2.200000,1.261564e+05,7.186760e+04,155.0
102677,dhtz0k6x,0,0,3.025200e+05,118690.0,1.000000,2.000000,3.923150e+05,9.932300e+04,2.0
15992,dhtwz2fw,1,0,NaN,NaN,0.636752,3.594017,9.306420e+04,5.812242e+04,16146.0


In [22]:
first_stage_model=first_stage_regression(analysis_df)


================= First-Stage Regression Results (OLS) =================
                 Variable      Coef   StdErr          t    p_value
                    const  0.014627 0.000709  20.631939 0.00000000
          peer_avg_damage  0.191874 0.001145 167.570398 0.00000000
peer_avg_BldgValue_scaled -0.000191 0.000026  -7.389728 0.00000000
peer_avg_LandValue_scaled  0.006865 0.000275  24.971371 0.00000000
          DamageLevel_num  0.037284 0.000633  58.901409 0.00000000
         BldgValue_scaled  0.000070 0.000038   1.862499 0.06253595
F-statistic for instrument (peer_avg_damage): 28079.838
p-value: 0.00000000


In [24]:
second_stage_model = second_stage_regression(analysis_df, first_stage_model)


================= Second-Stage Regression Results (IV-2SLS) =================
          Variable          Coef       StdErr          t    p_value
             const  2.396781e-01 1.787563e-03 134.080920 0.00000000
         BldgValue -5.132172e-09 2.723633e-09  -1.884311 0.05952290
   DamageLevel_num  7.076541e-02 1.713463e-03  41.299632 0.00000000
peer_avg_BldgValue  2.421521e-09 3.389376e-10   7.144446 0.00000000
peer_avg_LandValue -2.511790e-08 8.290149e-09  -3.029849 0.00244676
 peer_avg_decision  4.417725e-01 1.615271e-02  27.349737 0.00000000
F-statistic (overall): 3626.467
p-value: 0.00000000
